## Preparation

In [1]:
# To update ubuntu to be able to run apt install correctly
!apt-get update &> /dev/null

In [2]:
# Install chromium browser and web driver
!apt-get install chromium-browser &> /dev/null
!apt install chromium-chromedriver &> /dev/null

In [3]:
# Install python libraries
!pip install selenium &> /dev/null

In [4]:
import os
import time
import uuid
import pandas as pd
from google.colab import files
from google.colab import drive
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Options to disable chromium functionalities
options = webdriver.ChromeOptions()
options.add_argument('--verbose')
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920, 1200')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options = options)

In [12]:
# "physically" scroll to cretain attribute within a website
def scroll_to_element(driver, element):
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()

## Data scrap

In [ ]:
## Frist making a dataframe with the list of countries + country link to avoid "stale selenium stuff"
home_link = "https://visaindex.com/"
driver.get(home_link)
time.sleep(2)

countries_list = []
countries_list_sel = driver.find_elements(By.XPATH, '//div[@class="list-group list-group-flush"]//div[@class="countryContainer"]//div[@class="countryElement"]')

for country in countries_list_sel:
  country_name = country.text
  country_link = country.find_element(By.XPATH, './/a').get_attribute('href')

  countries_list.append({
      'country_name': country_name,
      'country_link': country_link
  })

passport_index = []

## Then making the real data scrap
for country in countries_list:
  time.sleep(1)
  country_name = country['country_name']
  country_link = country['country_link']

  print(country_name,"="*100, country_link)

  driver.get(country_link)
  time.sleep(1)

  passport_description = driver.find_element(By.XPATH, '//div[@class="topContainer"]//div[@class="d-flex mainTopContainer"]/p').text
  passport_image = driver.find_element(By.XPATH, '//div[@class="topContainer"]//div[@class="passportImage"]/img').get_attribute('src')

  clearance_container = driver.find_elements(By.XPATH, '//div[@class="row justify-content-lg-center requirementsLists"]/div[contains(@class, "countries-col")]')

  for clearance_box in clearance_container:
    # Find the clearance type title
    clearance_type = clearance_box.find_element(By.XPATH, './/div[@class="col-title text-secondary font-weight-bold display-8"]')

    # Find the container with the countries
    countries = clearance_box.find_element(By.XPATH, './/div[@class="list-group borderLight bg-white"]')
    countries = countries.find_elements(By.XPATH, './/a')

    for country_detail in countries:
        print(clearance_type.text, "="*20,country_detail.text, passport_image)

        passport_index.append({
            'passport': country_name,
            'passport_link': country_link,
            'passport_description': passport_description,
            'passport_image': passport_image,
            'target_country': country_detail.text,
            'clearance_type': clearance_type.text
        })


passport_index_dataset = pd.DataFrame(passport_index)
filename = 'passport_index'
passport_index_dataset.to_json(f'/content/drive/My Drive/data scrap/passport index/{filename}.json', orient='records', lines=True)
passport_index_dataset.to_csv(f'/content/drive/My Drive/data scrap/passport index/{filename}.csv', index=False)
passport_index_dataset.to_excel(f'/content/drive/My Drive/data scrap/passport index/{filename}.xlsx', index=False)

print("="*150)
print("="*150,"FILES SAVED TO G-DRIVE")
print("="*150)